In [1]:
import os
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm.sklearn import LGBMClassifier
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb
from evaluation.metrics import evaluate
from tqdm import tqdm
import torch
from util import ThresholdClassifier

# Single Step With Prior

In [8]:
from sklearn.metrics import f1_score


def eval_single_step(model_name, with_prior=True):
    model_path = os.path.join('../storage', "creditcard", model_name)
    if not os.path.exists(model_path):
        return None

    test_set = pd.read_csv(os.path.join(model_path, 'test.csv'))
    dev_set = pd.read_csv(os.path.join(model_path, 'val.csv'))
    if not with_prior:
        if 'fst_step_pred' in test_set.columns:
            test_set['pred'] = test_set['fst_step_pred']
            dev_set['pred'] = dev_set['fst_step_pred']
        else:
            # if not within the range of [0, 1], then use logit to transform
            if test_set['fst_step_scores'].max() > 1 or test_set['fst_step_scores'].min() < 0:
                import math
                def sigmoid(x):
                    return 1 / (1 + math.exp(-x))
                test_set['fst_step_scores'] = test_set['fst_step_scores'].apply(sigmoid)

            if dev_set['fst_step_scores'].max() > 1 or dev_set['fst_step_scores'].min() < 0:
                dev_set['fst_step_scores'] = dev_set['fst_step_scores'].apply(sigmoid)
    
    test_set['pred_proba'] = test_set['fst_step_scores']
    dev_set['pred_proba'] = dev_set['fst_step_scores']
    eval_dict = evaluate(test_set, with_prior=with_prior, dev_set=dev_set)
    result_df = pd.DataFrame(eval_dict, index=[0])
    # no column limit
    pd.set_option('display.max_columns', None)
    # print(result_df)

    return eval_dict

In [11]:
all_results = []
for fold in [1, 2, 3]:
    model_name = [
        f'rf_benchmark_fold{fold}', f'xgb_benchmark_fold{fold}', f'lgbm_benchmark_fold{fold}',
        f'deepsad_benchmark_fold{fold}', f'deepisolationforest_benchmark_fold{fold}', f'feawad_benchmark_fold{fold}', f'slad_benchmark_fold{fold}',
        f'sour_100_binary_fold{fold}', f'tabtransformer_benchmark_fold{fold}',
        f'lambdamart_100_binary_1000_trees_fold{fold}', f'lambdamart_100_binary_10000_trees_fold{fold}', f'lambdamart_pabce_10000_fold{fold}'
    ]
    proposed_model_names = [
        f'rankformer_100_binary_softmax_fold{fold}', f'rankformer_100_binary_lambdaloss_fold{fold}',
        f'pariskranker_100_binary_graph_fold{fold}'
    ]
    for proposed_model_name in proposed_model_names:
        model_path = os.path.join('../storage', "creditcard", proposed_model_name)
        if not os.path.isdir(model_path):
            continue
        subfolders = [f"{proposed_model_name}/{f.name}" for f in os.scandir(model_path) if f.is_dir()]
        model_name.extend(subfolders)

    for name in tqdm(model_name):
        eval_dict = eval_single_step(name, with_prior=True)
        if eval_dict is not None:
            eval_dict["model"] = name.rsplit('_fold', 1)[0]  # Strip fold suffix
            eval_dict["fold"] = fold
            all_results.append(eval_dict)

# Combine and compute stats
df_all = pd.DataFrame(all_results)
grouped = df_all.groupby("model").agg(['mean'])

# Flatten column names
grouped.columns = ['_'.join(col).strip() for col in grouped.columns.values]
grouped = grouped.sort_values(by='f1_score_mean', ascending=False)

grouped

100%|██████████| 15/15 [00:04<00:00,  3.74it/s]


,f1_score_mean,PnL_mean,auc_score_mean,precision_mean,sensitivity_mean,specificity_mean,fold_mean
model,,,,,,,
pariskranker_100_binary_graph,0.987033,-3.136839e+04,0.999800,0.974300,0.974300,0.999767,2.0
rankformer_100_binary_softmax,0.982000,-4.382178e+04,0.999933,0.964400,0.964400,0.999633,2.0
deepsad_benchmark,0.976667,-5.249995e+04,0.999867,0.953867,0.953867,0.999533,2.0
xgb_benchmark,0.975500,-5.814926e+04,0.999900,0.951567,0.951567,0.999500,2.0
lgbm_benchmark,0.972900,-6.365904e+04,0.999900,0.946300,0.946300,0.999500,2.0
lambdamart_100_binary_10000_trees,0.969367,-7.281716e+04,0.999900,0.939300,0.939300,0.999367,2.0
lambdamart_100_binary_1000_trees,0.969367,-7.275735e+04,0.999900,0.939300,0.939300,0.999367,2.0
rf_benchmark,0.968167,-7.477060e+04,0.999833,0.937000,0.937000,0.999367,2.0
lambdamart_pabce_10000,0.966667,-8.389595e+04,0.999067,0.934033,0.934033,0.999333,2.0


In [12]:
all_results = []
for fold in [1, 2, 3]:
    model_name = [
        f'rf_benchmark_fold{fold}', f'xgb_benchmark_fold{fold}', f'lgbm_benchmark_fold{fold}',
        f'deepsad_benchmark_fold{fold}', f'deepisolationforest_benchmark_fold{fold}', f'feawad_benchmark_fold{fold}', f'slad_benchmark_fold{fold}',
        f'sour_100_binary_fold{fold}', f'tabtransformer_benchmark_fold{fold}',
        f'lambdamart_100_binary_1000_trees_fold{fold}', f'lambdamart_100_binary_10000_trees_fold{fold}', f'lambdamart_pabce_10000_fold{fold}'
    ]
    proposed_model_names = [
        f'rankformer_100_binary_softmax_fold{fold}', f'rankformer_100_binary_lambdaloss_fold{fold}',
        f'pariskranker_100_binary_graph_fold{fold}'
    ]
    for proposed_model_name in proposed_model_names:
        model_path = os.path.join('../storage', "creditcard", proposed_model_name)
        if not os.path.isdir(model_path):
            continue
        subfolders = [f"{proposed_model_name}/{f.name}" for f in os.scandir(model_path) if f.is_dir()]
        model_name.extend(subfolders)

    for name in tqdm(model_name):
        eval_dict = eval_single_step(name, with_prior=False)
        if eval_dict is not None:
            eval_dict["model"] = name.rsplit('_fold', 1)[0]  # Strip fold suffix
            eval_dict["fold"] = fold
            all_results.append(eval_dict)

# Combine and compute stats
df_all = pd.DataFrame(all_results)
grouped = df_all.groupby("model").agg(['mean'])

# Flatten column names
grouped.columns = ['_'.join(col).strip() for col in grouped.columns.values]
grouped = grouped.sort_values(by='f1_score_mean', ascending=False)

grouped

100%|██████████| 15/15 [00:12<00:00,  1.22it/s]


,f1_score_mean,PnL_mean,auc_score_mean,precision_mean,sensitivity_mean,specificity_mean,fold_mean
model,,,,,,,
pariskranker_100_binary_graph,0.985633,-3.533236e+04,0.999800,0.970767,0.972567,0.999733,2.0
rankformer_100_binary_softmax,0.981133,-4.534284e+04,0.999933,0.958067,0.967300,0.999567,2.0
lgbm_benchmark,0.972233,-6.627772e+04,0.999900,0.968167,0.922933,0.999700,2.0
xgb_benchmark,0.971800,-6.764460e+04,0.999900,0.975833,0.914767,0.999767,2.0
lambdamart_100_binary_10000_trees,0.969867,-7.253309e+04,0.999900,0.956500,0.924700,0.999600,2.0
lambdamart_100_binary_1000_trees,0.967900,-7.711850e+04,0.999900,0.941933,0.931700,0.999400,2.0
rf_benchmark,0.964867,-8.567236e+04,0.999833,0.968333,0.896067,0.999733,2.0
tabtransformer_benchmark,0.959333,-8.903083e+04,0.997100,0.900733,0.939267,0.998967,2.0
rankformer_100_binary_lambdaloss,0.953300,-1.040376e+05,0.999733,0.888300,0.928767,0.998800,2.0
